# 仮想カメラ

ディジタル画像の撮像過程「「仮想カメラ」を作る」

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd "/content/drive/My Drive/Colab Notebooks/sample/"

3次元モデル（plyファイル）の読み込み。超簡易版

In [ ]:
import csv

with open('cube.ply') as f:
  reader = csv.reader(f, delimiter=' ')
  l = [row for row in reader]

cube = np.array(l[10:]).astype(float) # ヘッダを無視する

In [ ]:
print(cube.shape)
print(cube)

投影

In [ ]:
import math

# 準備
img = np.zeros((480,640)) # 640x480の画像
fov_x = 60.0  # 水平画角

# パラメータの計算
w = img.shape[1]  # 画像の横幅
h = img.shape[0]  # 画像の高さ
cx = w / 2  # カメラの光学中心が画像の中心に来るようにする
cy = h / 2  # カメラの光学中心が画像の中心に来るようにする
f = w / (2 * math.tan(math.radians(fov_x/2.0)))

A = np.array([[f, 0, cx],
             [0, f, cy],
             [0, 0, 1]])

In [ ]:
print(A)

In [ ]:
suv = (A@cube.T).T            # s(u,v,1) = A@(X,Y,Z)
uv = suv[:,:2] / suv[:,2:]    # (u,v) = s(u,v,1) / s

In [ ]:
vu = uv[:,[1,0]]       # uとvを入れ替え
vu_index = [tuple(x) for x in vu.astype(int)] # 整数のtupleにした

for ind in vu_index:
  if 0 <= ind[0] <= (h-1) and 0 <= ind[1] <= (w-1): # 画像をはみ出ない場合
    img[ind] = 255

cv2_imshow(img)